In [4]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from mediapipe import solutions

from utils import *
from data_utils import prepare_dataframe, generate_dataframe
from custom_pose_landmarks import CustomPoseLandmark
from canonical_form import PoseCanonicalForm

In [5]:
# Selected values of pose landmarks corresponding to PoseLandmark class from MediaPipe library
values = [0, 11, 12, 13, 14, 15, 16, 19, 20, 23, 24, 25, 26, 27, 28, 31, 32]

# Custom pose landmark names
landmarks = {
    'THORAX': ['NOSE'],
    'PELVIS': ['LEFT_HIP', 'RIGHT_HIP'],
}

# MediaPipe solutions
mp_drawing = solutions.drawing_utils
mp_pose = solutions.pose


pose = CustomPoseLandmark(mp_pose, values, landmarks)